In [12]:
from autogen import ConversableAgent
from openai import OpenAI
from pathlib import Path
from LLM_configs import *
from Prompt_dics import *

In [2]:
# 加载 .env 文件
load_dotenv()

True

## 文件提取

In [7]:
# Kimi的文件接口
client = OpenAI(
    api_key = os.environ.get("Kimi_API_KEY"),
    base_url = "https://api.moonshot.cn/v1",
)

In [8]:
# 使用kimi api读取
def file_extraction(file_path):
    file_object = client.files.create(file=Path(file_path), purpose="file-extract")
    file_content = client.files.content(file_id=file_object.id).text
    return file_content

In [9]:
text = file_extraction("test files/象上-文稿-转写结果.docx")

In [10]:
PPT = file_extraction("test files/上海象上投资.pptx")

In [13]:
requirements = Strategy_requirements_dic['Simple Quant Strategy']

## Agent设计

In [14]:
def report_writing(weak_config, strong_config):
    # 定义强弱模型助手
    
    extraction_assistant = ConversableAgent(
        name = "信息提取员",
        system_message = "你是一个信息提取员，任务是将我需要的信息，从我给你的一大段文字中提取出来，并返回给我",
        llm_config = weak_config
    )
    
    integrate_assistant = ConversableAgent(
        name="报告撰写员",
        system_message=f"你是一个报告撰写员，需要根据我发给你的字段，根据我的要求，详细的写出一份报告，并返回给我,以下是要求：不需要面面俱到，写出提供材料中包含的即可：{requirements}",
        llm_config = strong_config
    )
    
    # 开始分析
    quant_idea = extraction_assistant.generate_reply(messages=[{"content": text, "role":"system"},
                                                      {"content": "管理⼈量化策略的整体思路, 例如，是偏强逻辑为主, 对因⼦的可解释性有很⾼的要求；还是偏暴力挖掘为主，使用机器学习较多", "role": "user"}])
    
    factor_composition = extraction_assistant.generate_reply(messages=[{"content": text, "role":"system"},
                                                      {"content": "从文件中提取出管理人的策略中的因子组成和因子数量,包括量价类,基本面，另类因子占比多少，以及因子的总数", "role": "user"}])
    
    factor_explanation = extraction_assistant.generate_reply(messages=[{"content": text, "role":"system"},

                                                      {"content": "从文件中提取出管理人对其不同因子类型的解释（如有），具体来说主要是不同因子是基于哪些数据锝出的，不要编造，只能抽取提供信息中有的内容。如果没有详细说明，则返回无", "role": "user"}])
    
    model_explanation = extraction_assistant.generate_reply(messages=[{"content": text, "role":"system"},
                                                      {"content": "从文件中提取出管理人的建模方法，需要明确提示是线性还是机器学习模型或者是两者都有及其偏好，详述他们的建模方法论, 并提取出他们使用了的算法和用途", "role": "user"}])
    
    factor_update = extraction_assistant.generate_reply(messages=[{"content": text, "role":"system"},
                                                      {"content": "从文件中提取出管理人的新因子入库条件，例如：会在模拟盘中观察 X ⽉, 因⼦相关性不⾼于XX, 关注 ICIR/夏普等", "role": "user"}])
    
    strategy_excecution= extraction_assistant.generate_reply(messages=[{"content": text, "role":"system"},
                                                      {"content": "从文件中提取出管理人的具体操作，例如有无使用自研/外部的下单算法，不同策略的持股数是多少，年化双边换手率多少倍等", "role": "user"}])
    
    # 撰写文章
    components_list = [quant_idea, factor_composition, factor_explanation, model_explanation, factor_update,
                       strategy_excecution]


    final_report = integrate_assistant.generate_reply(messages=[{"content": str(components_list), "role": "system"},
                                                                {
                                                                    "content": "按需求生成报告，不要分段，写成一整段，要求逻辑清晰，细节完整，长度越长越好，但不要有过多的你自由发挥的解释，客观记录为主, 不要出现评价语言，例如“这体现了XXXX",
                                                                    "role": "user"}])
    cost_list = [extraction_assistant.get_total_usage(),integrate_assistant.get_total_usage()]
    
    return quant_idea, factor_composition, factor_explanation, model_explanation, factor_update, strategy_excecution,cost_list,final_report
    
    

In [75]:
def report_writing2(weak_config, strong_config):
    # 定义强弱模型助手
    
    extraction_assistant = ConversableAgent(
        name = "信息提取员",
        system_message = "你是一个信息提取员，任务是将我需要的信息，从我给你的一大段文字中提取出来，并返回给我",
        llm_config = weak_config
    )
    
    integrate_assistant = ConversableAgent(
        name="报告撰写员",
        system_message=f"你是一个报告撰写员，需要根据我发给你的字段，根据我的要求，详细的写出一份报告，并返回给我,以下是要求：不需要面面俱到，写出提供材料中包含的即可：{requirements}",
        llm_config = strong_config
    )
    
    # 开始分析
    content = extraction_assistant.generate_reply(messages=[{"content": text, "role":"system"},
                                                            {"content": "文字需要尽可能的长，每个部分需要包含所有的细节，不能有大量的信息缺失", "role":"system"},
                                                      {"content": 
                                                           "1. 管理⼈量化策略的整体思路, 例如，是偏强逻辑为主, 对因⼦的可解释性有很⾼的要求；还是偏暴力挖掘为主，使用机器学习较多。 2.从文件中提取出管理人的策略中的因子组成和因子数量。3. 从文件中提取出管理人对其不同因子类型的解释（如有），具体来说主要是不同因子是基于哪些数据锝出的，不要编造，只能抽取提供信息中有的内容。如果没有详细说明，则返回无。4.从文件中提取出管理人的建模方法，需要明确提示是线性还是机器学习模型或者是两者都有及其偏好，详述他们的建模方法论。若提到算法，需要明确指出使用了什么算法，怎么用的。5.从文件中提取出管理人的新因子入库条件. 6.从文件中提取出管理人的具体操作，例如有无使用自研/外部的下单算法，不同策略的持股数是多少，年化双边换手率多少倍"
                                                           , "role": "user"}])


    final_report = integrate_assistant.generate_reply(messages=[{"content": content, "role": "system"},
                                                                {
                                                                    "content": "按需求生成报告，不要分段，写成一整段，要求逻辑清晰，细节完整，长度越长越好，但不要有过多的你自由发挥的解释，客观记录为主,不要出现评价语言，例如“这体现了XXXX",
                                                                    "role": "user"}])
    cost_list = [extraction_assistant.get_total_usage(),integrate_assistant.get_total_usage()]
    
    return content, cost_list,final_report
    
    

In [19]:
quant_idea, factor_composition, factor_explanation, model_explanation, factor_update, strategy_excecution, cost_list, Ali_result =report_writing(Qwen_weak_llm_config,Qwen_strong_llm_config)


>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...


In [16]:
factor_explanation

'在提供的文本中，管理人对其不同因子类型的解释主要涉及以下几个方面：\n\n1. 因子的类型：管理人提到了基本面因子、另类因子和量价因子。其中，基本面因子占比40%，另类因子占比10%，量价因子占比50%。\n\n2. 因子的挖掘：管理人表示，他们的因子主要是手工挖掘出来的，并且有一位同事正在做机器学习的部分。\n\n3. 因子的逻辑性：管理人强调，他们的因子都是有逻辑的因子。\n\n4. 因子的入库条件：管理人提到，在选择因子的过程中，会先进行单因子测试，确保因子在多空收益、夏普比等数值上达到一定要求后，才会将其放入数据库。\n\n5. 因子的复权和组合：管理人提到，他们会结合因子的相关性和对组合的贡献，将因子加入到实盘策略中。\n\n6. 因子的权重调整：管理人表示，因子的权重会基于历史表现和因子之间的相关性进行调整。\n\n7. 因子的出库：管理人提到，他们会持续跟踪因子的表现，并根据其与历史表现的相符性，对因子的权重进行相应的调整。\n\n8. 数据来源：管理人提到，他们使用的量化数据主要来自通联，而不是同花顺。\n\n9. 舆情系统：管理人提到，他们有一个自己开发的舆情系统，用于抓取不同平台的信息，但目前尚未用到实盘策略中。\n\n10. 机器学习的应用：管理人提到，他们使用机器学习主要是为了预测不同子策略的表现，并据此调整权重。\n\n这些信息提供了对管理人如何基于不同数据得出因子的一定理解，但并没有详细说明每个因子具体是基于哪些数据得出的。'

In [20]:
print(Ali_result)

管理人的量化策略整体倾向于逻辑性和可解释性，核心思路在于手工挖掘有效因子并确保其背后逻辑的合理性，同时积极探索机器学习技术以增强策略效能。策略中因子组成百分比如下：基本面因子占40%，另类因子占10%，量价因子占50%，因子库总计约含三五十个因子，实际运用中精选约30个因子。因子挖掘主要依靠手工完成，重视因子的逻辑基础，同时也有一名团队成员致力于将机器学习融入因子发现过程，但目前该方法尚处于研究阶段。

因子处理方面，管理人通过严格测试确保因子的有效性，包括多空收益、特定指标检验，并根据因子的历史表现和相互间相关性调整权重。在因子合成与处理中，采用基于历史回溯和相关性分析的方法论，尽管具体算法未详细阐述，但强调了对因子逻辑性和有效性的持续评估。模型构建上，管理人虽偏向于手工挖掘因子构建的线性模型，但也初步涉足机器学习领域，用于预测策略表现和动态调整策略权重，展现了策略灵活性和对技术前沿的开放态度。

新因子入库标准严格，需通过单因子测试验证其收益能力，评估与现有因子的相关性以避免冗余，同时在模拟环境中验证其对组合的正面贡献，模拟测试周期依据因子类型从两个月至半年不等。执行层面，管理人采用外部T0增强服务和悦然系统优化交易，每个产品平均持有500只股票，覆盖全市场，年化双边换手率在60至80倍之间，此数据不包含T0交易部分。至于对冲方法，当前信息未直接提及具体的对冲手段或是否有敞口存在，表明管理人可能更侧重于选股策略本身的优化而非依赖对冲机制来控制风险。


In [18]:
cost_list

[{'total_cost': 0.451836,
  'moonshot-v1-8k': {'cost': 0.451836,
   'prompt_tokens': 35904,
   'completion_tokens': 1749,
   'total_tokens': 37653}},
 {'total_cost': 0.14748,
  'qwen-max': {'cost': 0.14748,
   'prompt_tokens': 2367,
   'completion_tokens': 440,
   'total_tokens': 2807}}]

In [31]:
strategy_excecution

'从提供的内容中，可以提取出以下与管理人具体操作相关的信息：\n\n### 使用的下单算法\n- **外部下单算法**: 使用了第三方的T0增强服务。\n- **具体系统**: 使用悦然的系统来进行T0增强。\n\n### 不同策略的持股数\n- **总持股数**: 每个产品大约持有500只股票。\n- **持股范围**: 全市场选股。\n\n### 年化双边换手率\n- **换手率**: 60~80倍。\n- **备注**: 这个换手率不包括T0部分。\n\n以上信息概括了管理人在下单算法的选择、不同策略下的持股数量以及年化双边换手率等方面的具体操作。'

In [89]:
content, cost_list,final_report = report_writing2(kimi_weak_llm_config,Qwen_strong_llm_config)


>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...


In [90]:
print(final_report)

管理人的量化策略核心在于强逻辑驱动，着重于因子的高可解释性，以此为基础构建投资组合。策略框架内，共集成约200个因子，其中量价类因子占比30%，基本面类因子占据40%，而另类因子，如公司公告、新闻舆情、专利信息等，占比达到30%。在因子挖掘方面，管理人采取以手工挖掘为主的方式，相对减少对机器学习技术的依赖，保持策略的人工智慧介入度与逻辑透明性。

对于因子处理，管理人运用机器学习技术进行因子的智能组合与分组，确保因子间的高效协同，同时实施行业中性和市值中性策略，以降低特定风险暴露。模型构建上，管理人灵活结合线性模型与机器学习模型，依据因子特性和市场环境，分别利用线性回归模型进行基础因子评估，以及通过神经网络等高级机器学习算法捕捉复杂非线性关系，偏好模型选择取决于因子特性和预测目标。

在因子入池标准上，管理人严格要求候选因子需在模拟环境中观察数月至满足特定标准，如因子间相关性不超过预设阈值，以及通过ICIR（信息比率）和夏普比率等指标的严格筛选，确保因子的有效性和稳定性。

执行层面，管理人对股票实施多维度分析后，采用算法下单优化交易成本与执行效率，最终维持在一个宽基持股状态，持股数量通常在数百至千只不等，旨在分散风险并捕捉广泛市场机会。对冲策略上，若适用，管理人可能采用股指期货或其他金融衍生品进行风险对冲，控制敞口，确保策略稳健运行。

持仓特征方面，管理人通常持有大量个股，广泛覆盖指数成分股，实现市场广泛参与的同时，注重个股在指数内的权重分布，以维持组合的市场代表性。

交易风格上，管理人倾向于保持中等到较高活跃度，年化双边换手率落在15至25倍区间，既保证策略的灵活性，又控制了交易成本，实现了策略收益与流动性的平衡。


In [82]:
print(content)

1. 管理人量化策略的整体思路：
   管理人的量化策略主要侧重于基本面量化，强调逻辑性和可解释性。他们使用大量的基本面信息，结合小部分量价信息构建投资组合。策略中，基本面因子占主导地位，量价因子作为辅助。管理人对因子的逻辑性有较高要求，注重因子与市场环境、公司报表情况的关联性。

2. 管理人的策略中的因子组成和因子数量：
   管理人的因子库包括基本面因子和量价因子。基本面因子大约有三四百个，实际使用的可能在五六十个左右。量价因子的库中可能有几千个，但实际使用的数量不详。

3. 管理人对其不同因子类型的解释：
   - 基本面因子：主要基于财务报表、公司公告等与公司基本面相关的信息。这些信息被整理成特征，用于机器学习训练。
   - 量价因子：虽然具体解释不详，但提到量价因子的挖掘可能使用遗传算法，通过统计学指标完成。

4. 管理人的建模方法：
   管理人的建模方法包括线性和机器学习模型。他们使用非线性机器学习训练方式，将基本面因子和量价因子一起投入模型，让模型自行处理。此外，量价因子的挖掘可能使用遗传算法。

5. 管理人的新因子入库条件：
   新因子入库时，管理人会参考因子的SCI（统计显著性检验）结果，并根据当前库内因子的特征进行交叉验证。他们偏好具有一定替代性或负相关性的因子。

6. 管理人的具体操作：
   - 下单算法：管理人没有自研交易算法，完全依靠券商提供的算法。
   - 持股数：管理人的一篮子持仓大约有500~800只个股。
   - 年化双边换手率：管理人的换手率大约在15~25倍之间。


In [83]:
cost_list

[{'total_cost': 0.094752,
  'moonshot-v1-8k': {'cost': 0.094752,
   'prompt_tokens': 7500,
   'completion_tokens': 396,
   'total_tokens': 7896}},
 {'total_cost': 0.09884000000000001,
  'qwen-max': {'cost': 0.09884000000000001,
   'prompt_tokens': 980,
   'completion_tokens': 497,
   'total_tokens': 1477}}]